# Environment setup

In [1]:
!npm install vietnamese-stopwords

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ vietnamese-stopwords@0.0.2
updated 1 package and audited 1 package in 0.851s
found 0 vulnerabilities



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd '/content/drive/MyDrive/CustomerReviewSentiment'

/content/drive/MyDrive/CustomerReviewSentiment


In [4]:
!pip install sentence_transformers

# Data preparation

In [12]:
import pandas as pd

data = pd.read_csv('clean_data.csv')

In [13]:
data

,Unnamed: 0,content,score,thumbsUpCount,Application
0,0,"rất thuận tiện , có lợi ích .",5,2,Messenger
1,1,tốt,4,213,Messenger
2,2,ok,5,1,Messenger
3,3,"xài rất tốt , nên tải nha",5,8,Messenger
4,4,gútttt,5,1,Messenger
...,...,...,...,...,...
310741,99995,ok,5,0,Instagram
310742,99996,g o o d,5,0,Instagram
310743,99997,có,5,0,Instagram
310744,99998,nhiều lúc tôi k lấy đc hiệu ứng,1,0,Instagram


In [14]:
print(data.columns[0])
data = data.drop(data.columns[0], axis=1)
data


Unnamed: 0


,content,score,thumbsUpCount,Application
0,"rất thuận tiện , có lợi ích .",5,2,Messenger
1,tốt,4,213,Messenger
2,ok,5,1,Messenger
3,"xài rất tốt , nên tải nha",5,8,Messenger
4,gútttt,5,1,Messenger
...,...,...,...,...
310741,ok,5,0,Instagram
310742,g o o d,5,0,Instagram
310743,có,5,0,Instagram
310744,nhiều lúc tôi k lấy đc hiệu ứng,1,0,Instagram


#Text Preprocessing with NLTK

In [15]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('wordnet')

def read_stop_words(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stop_words = [line.strip() for line in file]
    return stop_words

stop_words_path = 'vietnamese-stopwords.txt'
stop_words = set(read_stop_words(stop_words_path))

lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if not isinstance(text, str):
        return ''

    text = re.sub(r'[^\w\s]', '', text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

data['content'] = data.content.apply(lambda x: clean_text(x))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
def count_sentences(text):
    sentences = text.split(".")
    return len(sentences)

data['sentence_count'] = data['content'].apply(count_sentences)
data = data[data['sentence_count'] <= 1].drop(columns=['sentence_count'])
data = data[0:1000]

In [17]:
data

,content,score,thumbsUpCount,Application
0,thuận tiện lợi ích,5,2,Messenger
1,,4,213,Messenger
2,ok,5,1,Messenger
3,xài tải nha,5,8,Messenger
4,gútttt,5,1,Messenger
...,...,...,...,...
995,cập nhật clllll xóa mẹ mess đi,1,1,Messenger
996,ứng dụng kém,1,0,Messenger
997,tiếng vọng,1,0,Messenger
998,ok,5,0,Messenger


#Feature Extraction

In [31]:
from sentence_transformers import SentenceTransformer
import pandas as pd
encode_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
encode_model.eval()
encoded_values = encode_model.encode(list(data['content']), batch_size = 1000)
encoded_df = pd.DataFrame(encoded_values, columns=[f'feature_{i}' for i in range(encoded_values.shape[1])])
data = pd.concat([data, encoded_df], axis=1)
data

,content,score,thumbsUpCount,Application,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
0,thuận tiện lợi ích,5,2,Messenger,0.007724,-0.029080,0.013690,-0.009045,0.023238,-0.000431,...,-0.016954,-0.036230,-0.035321,0.018905,-0.004153,-0.050888,0.026344,-0.036549,0.005494,-0.022665
1,,4,213,Messenger,0.017581,-0.002738,-0.038254,-0.041900,-0.053605,0.027044,...,0.016105,-0.003971,0.019439,0.017352,-0.038702,-0.057055,-0.019953,-0.015060,0.021243,-0.026957
2,ok,5,1,Messenger,0.028191,-0.010388,-0.052607,-0.094258,0.026439,0.025329,...,0.044415,0.029121,0.023854,0.024050,0.003137,-0.067067,-0.010747,0.015472,-0.007338,-0.064763
3,xài tải nha,5,8,Messenger,-0.012584,-0.066111,-0.015364,-0.006006,-0.049788,0.031777,...,0.029615,-0.012054,0.038802,0.006297,0.011572,-0.025683,0.032493,-0.004576,0.034961,-0.028953
4,gútttt,5,1,Messenger,0.011783,-0.014584,-0.033242,-0.039972,-0.041038,0.000820,...,0.018747,-0.020979,0.011736,0.006028,-0.020612,-0.021013,0.004732,-0.011790,-0.021903,0.000539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,cập nhật clllll xóa mẹ mess đi,1,1,Messenger,-0.003216,-0.013464,0.004712,0.002270,-0.053654,-0.008947,...,0.005085,-0.007261,0.017898,0.018807,-0.013562,-0.015285,-0.032608,0.012056,0.017730,-0.023654
996,ứng dụng kém,1,0,Messenger,0.003408,0.011238,-0.003270,0.009822,0.010839,0.013731,...,0.005113,-0.001555,0.023239,0.026508,-0.008224,-0.027951,-0.023829,-0.005397,-0.006893,0.011582
997,tiếng vọng,1,0,Messenger,0.009905,0.016094,-0.045669,-0.000068,-0.040511,0.034474,...,0.018995,-0.012872,0.006261,-0.016766,-0.041759,-0.059727,-0.005747,-0.008027,0.004528,-0.001835
998,ok,5,0,Messenger,0.028191,-0.010388,-0.052607,-0.094258,0.026439,0.025329,...,0.044415,0.029121,0.023854,0.024050,0.003137,-0.067067,-0.010747,0.015472,-0.007338,-0.064763


In [21]:
data

,content,score,thumbsUpCount,Application,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_502,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511
0,thuận tiện lợi ích,5,2,Messenger,0.007724,-0.029080,0.013690,-0.009045,0.023238,-0.000431,...,-0.016954,-0.036230,-0.035321,0.018905,-0.004153,-0.050888,0.026344,-0.036549,0.005494,-0.022665
1,,4,213,Messenger,0.017581,-0.002738,-0.038254,-0.041900,-0.053605,0.027044,...,0.016105,-0.003971,0.019439,0.017352,-0.038702,-0.057055,-0.019953,-0.015060,0.021243,-0.026957
2,ok,5,1,Messenger,0.028191,-0.010388,-0.052607,-0.094258,0.026439,0.025329,...,0.044415,0.029121,0.023854,0.024050,0.003137,-0.067067,-0.010747,0.015472,-0.007338,-0.064763
3,xài tải nha,5,8,Messenger,-0.012584,-0.066111,-0.015364,-0.006006,-0.049788,0.031777,...,0.029615,-0.012054,0.038802,0.006297,0.011572,-0.025683,0.032493,-0.004576,0.034961,-0.028953
4,gútttt,5,1,Messenger,0.011783,-0.014584,-0.033242,-0.039972,-0.041038,0.000820,...,0.018747,-0.020979,0.011736,0.006028,-0.020612,-0.021013,0.004732,-0.011790,-0.021903,0.000539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,cập nhật clllll xóa mẹ mess đi,1,1,Messenger,-0.003216,-0.013464,0.004712,0.002270,-0.053654,-0.008947,...,0.005085,-0.007261,0.017898,0.018807,-0.013562,-0.015285,-0.032608,0.012056,0.017730,-0.023654
996,ứng dụng kém,1,0,Messenger,0.003408,0.011238,-0.003270,0.009822,0.010839,0.013731,...,0.005113,-0.001555,0.023239,0.026508,-0.008224,-0.027951,-0.023829,-0.005397,-0.006893,0.011582
997,tiếng vọng,1,0,Messenger,0.009905,0.016094,-0.045669,-0.000068,-0.040511,0.034474,...,0.018995,-0.012872,0.006261,-0.016766,-0.041759,-0.059727,-0.005747,-0.008027,0.004528,-0.001835
998,ok,5,0,Messenger,0.028191,-0.010388,-0.052607,-0.094258,0.026439,0.025329,...,0.044415,0.029121,0.023854,0.024050,0.003137,-0.067067,-0.010747,0.015472,-0.007338,-0.064763


In [25]:
data_features = data.drop(columns=['Application', 'score', 'content']).values
data_features.shape

(1000, 513)

#Splitting the Dataset

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_features, data['score'], test_size=0.2, random_state=42)

#Training the XGBoost Classifier

In [27]:
import xgboost as xgb

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

model = xgb.XGBClassifier(max_depth=10, n_estimators=1000, learning_rate=0.01)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

#Making Predictions and Evaluating the Model

In [28]:
predictions = model.predict(X_test)

In [29]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.33      0.01      0.03        77
           2       0.40      0.12      0.19        16
           3       0.15      0.10      0.12        21
           4       0.06      0.24      0.10        17
           5       0.00      0.00      0.00        69

    accuracy                           0.04       200
   macro avg       0.16      0.08      0.07       200
weighted avg       0.18      0.04      0.05       200



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [30]:
from sklearn.metrics import roc_auc_score
y_test_encoded = le.fit_transform(y_test)
pred_prob = model.predict_proba(X_test)
print(y_test_encoded.shape)
print(predictions.shape)
auc = roc_auc_score(y_test_encoded, pred_prob, multi_class='ovr')
print('AUC: %.2f' % auc)


(200,)
(200,)
AUC: 0.70
